# <B> Training step examples </B>

## AutoReload

In [52]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. parameter store 셋팅

In [53]:
import boto3
from utils.ssm import parameter_store

In [54]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)
strPrefix = pm.get_params(key="PREFIX")

## 2. training job 만들기

In [55]:
import sagemaker
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

In [56]:
pipeline_session = PipelineSession()
model_image_uri = sagemaker.image_uris.retrieve("blazingtext", strRegionName)
objective_name = "validation:accuracy"
max_jobs = 6
max_parallel_jobs = 3

#### 2.1. estimator

In [57]:
estimator=Estimator(
    base_job_name="training",
    image_uri=model_image_uri,
    role=pm.get_params(key=strPrefix + "SAGEMAKER-ROLE-ARN"), 
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size=30,
    max_run=360000,
    input_mode= 'File',
    #sagemaker_session=self.pipeline_session
)

#### 2.2. hyperparamters

In [58]:
estimator.set_hyperparameters(
    mode="supervised",
    epochs=10,
    min_epochs=5, # Min epochs before early stopping is introduced
    early_stopping=True,
    patience=2,
    learning_rate=0.01,
    min_count=2, # words that appear less than min_count are discarded 
    word_ngrams=1, # the number of word n-gram features to use.
    vector_dim=16, # dimensions of embedding layer
) 

hyperparameter_ranges={
    'epochs': IntegerParameter(5, 50),
    'learning_rate': ContinuousParameter(0.005, 0.01),
    'min_count': IntegerParameter(0, 100),
    'vector_dim': IntegerParameter(32, 300),
    'word_ngrams': IntegerParameter(1, 3)
}

tuner = HyperparameterTuner(
    estimator, 
    objective_name,
    hyperparameter_ranges,
    max_jobs=max_jobs,
    max_parallel_jobs=max_parallel_jobs,
    base_tuning_job_name="hptune"
)

#### 2.3 training

In [59]:
tuner.fit(
    job_name = "hyper-param-tune",
    inputs={
        "train": TrainingInput(
            s3_data=pm.get_params(key=strPrefix + "TRAIN-DATA-PATH"),
            content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=pm.get_params(key=strPrefix + "VALID-DATA-PATH"),
            content_type="text/csv"
        )
    },
    wait=False
)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


### appendix

### pytorch esimator
* https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html <br>
* https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/sagemaker.pytorch.html